In [3]:
import re
import pymorphy2
from os import listdir, path
from tqdm import tqdm_notebook
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel
from gensim.corpora import Dictionary

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('russian')
print(sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sergt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'д

In [5]:
morph = pymorphy2.MorphAnalyzer()

In [7]:
corpus = []
dct = Dictionary()
for directory in tqdm_notebook(listdir('lenta_news_wo_author')):
    for file in listdir(path.join('lenta_news_wo_author', directory)):
        with open(path.join('lenta_news_wo_author', directory, file) , encoding = 'utf-8') as f:
            doc = re.findall(r'\w+', "".join(f.readlines()))
            doc = [morph.parse(word)[0].normal_form for word in doc if word not in sw]
            dct.add_documents([doc])
            corpus.append(dct.doc2bow(doc))

HBox(children=(IntProgress(value=0, max=6606), HTML(value='')))

In [8]:
dct.save('dct')

In [10]:
news = '''
Огромный айсберг, отколовшийся от Антарктиды в 1992 году, представляет угрозу для судоходства в районе южной оконечности Южной Америки (Чили). Ученые полагают, что этот кусок льда из Антарктики есть неоспоримое свидетельство глобального потепления. Ледяная гора размерами 50 на 17 миль (приблизительно 80 на 27 километров) находится примерно в 300 милях от южной оконечности Южной Африки - поблизости от основных траспорных морских путей. Большинство из них закрыты. За направлением движения опасного объекта, получившего кодовое наименование B-10A, постоянно следят - в Тихий или Атлантический океан он смещается. 
Айсберг движется неременным курсом со скоростью примерно 8-10 миль в сутки. 
Велика вероятность того, что он достигнет побережья аргентинского острова Тьерра дель Фьюго. 
Представитель Чилийского Института научных исследований Антарктики Даниэль Торре Наварро отметил, что глобальное потепление, без сомнения, способсвует ускоренному таянию льда. "Мы не впадаем в панику, однако эти симптомы свидетельствуют о том, что с природой действительно что-то происходит", - заявил он. '''
news = dct.doc2bow([morph.parse(word)[0].normal_form 
                    for word in re.findall(r'\w+', news.lower()) 
                    if word not in sw])

In [12]:
lda = LdaModel(corpus)  

In [13]:
[dct.get(i[0]) for i in lda.get_topic_terms(max(lda[news], key=lambda x: x[1])[0])]

['учёный',
 'который',
 'исследование',
 'результат',
 'земля',
 'специалист',
 'исследователь',
 'обнаружить',
 'такой',
 'это']

In [14]:
lda.save('lda_model')

In [15]:
lsi = LsiModel(corpus)  

In [16]:
[dct.get(int(i[0])) for i in lsi.show_topic(max(lsi[news], key=lambda x: x[1])[0])]

['год', 'news', 'в', 'который', 'это', 'e', 't', 'один', 'россия', 'свой']

In [17]:
lsi.save('lsi_model')